In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
import rioxarray as rxr
import glob
from datetime import datetime
import utm
from pyproj import Proj, Transformer
import os

In [68]:
times_GOES = pd.read_csv('../../tifs/GOES_times.csv')

In [69]:
times_GOES.value = (times_GOES.value/1000).apply(datetime.fromtimestamp)

In [70]:
times_GOES

,system:index,value,.geo
0,0,2022-12-31 22:46:17.300,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1,2023-01-01 10:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2,2023-01-01 22:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,3,2023-01-02 10:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,4,2023-01-02 22:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...
720,720,2023-12-28 10:46:17.100,"{""type"":""MultiPoint"",""coordinates"":[]}"
721,721,2023-12-28 22:46:17.100,"{""type"":""MultiPoint"",""coordinates"":[]}"
722,722,2023-12-29 10:46:17.200,"{""type"":""MultiPoint"",""coordinates"":[]}"
723,723,2023-12-29 22:46:17.200,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [71]:
times_Landsat = pd.read_csv('../../tifs/Landsat_times.csv')

In [72]:
times_Landsat.value = (times_Landsat.value/1000).apply(datetime.fromtimestamp)

In [73]:
times_Landsat

,system:index,value,.geo
0,0,2023-01-11 10:46:48.979,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1,2023-02-28 10:46:30.544,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2,2023-03-16 11:46:23.913,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,3,2023-04-01 11:46:13.064,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,4,2023-04-17 11:45:58.932,"{""type"":""MultiPoint"",""coordinates"":[]}"
5,5,2023-05-03 11:45:53.830,"{""type"":""MultiPoint"",""coordinates"":[]}"
6,6,2023-05-19 11:45:43.340,"{""type"":""MultiPoint"",""coordinates"":[]}"
7,7,2023-06-04 11:45:48.086,"{""type"":""MultiPoint"",""coordinates"":[]}"
8,8,2023-06-20 11:45:55.135,"{""type"":""MultiPoint"",""coordinates"":[]}"
9,9,2023-07-06 11:46:07.055,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [5]:
geotiff_list = glob.glob('../../tifs/*.tif')
geotiff_list

['../../tifs/GOES_Landsat_image_55.tif',
 '../../tifs/GOES_Landsat_image_41.tif',
 '../../tifs/GOES_Landsat_image_69.tif',
 '../../tifs/GOES_Landsat_image_103.tif',
 '../../tifs/GOES_Landsat_image_96.tif',
 '../../tifs/GOES_Landsat_image_82.tif',
 '../../tifs/GOES_Landsat_image_117.tif',
 '../../tifs/GOES_Landsat_image_7.tif',
 '../../tifs/GOES_Landsat_image_6.tif',
 '../../tifs/GOES_Landsat_image_83.tif',
 '../../tifs/GOES_Landsat_image_116.tif',
 '../../tifs/GOES_Landsat_image_102.tif',
 '../../tifs/GOES_Landsat_image_97.tif',
 '../../tifs/GOES_Landsat_image_68.tif',
 '../../tifs/GOES_Landsat_image_40.tif',
 '../../tifs/GOES_Landsat_image_54.tif',
 '../../tifs/GOES_Landsat_image_42.tif',
 '../../tifs/GOES_Landsat_image_56.tif',
 '../../tifs/GOES_Landsat_image_114.tif',
 '../../tifs/GOES_Landsat_image_81.tif',
 '../../tifs/GOES_Landsat_image_95.tif',
 '../../tifs/GOES_Landsat_image_100.tif',
 '../../tifs/GOES_Landsat_image_4.tif',
 '../../tifs/GOES_Landsat_image_5.tif',
 '../../tifs/G

In [6]:
def sort_func(s):
    return int(s.split('image_')[1].split('.tif')[0])

In [7]:
geotiff_list = sorted(geotiff_list, key=sort_func)
geotiff_list

['../../tifs/GOES_Landsat_image_0.tif',
 '../../tifs/GOES_Landsat_image_1.tif',
 '../../tifs/GOES_Landsat_image_2.tif',
 '../../tifs/GOES_Landsat_image_3.tif',
 '../../tifs/GOES_Landsat_image_4.tif',
 '../../tifs/GOES_Landsat_image_5.tif',
 '../../tifs/GOES_Landsat_image_6.tif',
 '../../tifs/GOES_Landsat_image_7.tif',
 '../../tifs/GOES_Landsat_image_8.tif',
 '../../tifs/GOES_Landsat_image_9.tif',
 '../../tifs/GOES_Landsat_image_10.tif',
 '../../tifs/GOES_Landsat_image_11.tif',
 '../../tifs/GOES_Landsat_image_12.tif',
 '../../tifs/GOES_Landsat_image_13.tif',
 '../../tifs/GOES_Landsat_image_14.tif',
 '../../tifs/GOES_Landsat_image_15.tif',
 '../../tifs/GOES_Landsat_image_16.tif',
 '../../tifs/GOES_Landsat_image_17.tif',
 '../../tifs/GOES_Landsat_image_18.tif',
 '../../tifs/GOES_Landsat_image_19.tif',
 '../../tifs/GOES_Landsat_image_20.tif',
 '../../tifs/GOES_Landsat_image_21.tif',
 '../../tifs/GOES_Landsat_image_22.tif',
 '../../tifs/GOES_Landsat_image_23.tif',
 '../../tifs/GOES_Landsat_

In [53]:
def to_binary_string(num):
    if np.isnan(num):
        return np.NaN
    else:
        return f'{int(num):b}'

In [10]:
to_binary_string(21952)

'101010111000000'

In [64]:
# Define UTM18N projection (WGS84 datum)
global utm_proj 
utm_proj = Proj(proj="utm", zone=18, datum="WGS84", northern=True)

In [65]:
"""
Processing of individual .tif files.

Performs a variety of tasks on the data to make it more easy to read and understand.

Attributes:
    tif (str): Path where tif file is located.
    time (datetime or str): Date and time of when the data was collected.
    name (str): Desired name of output file. Just the name, leave out the ".format" part at the end.
    coord_bounds (tuple or list, optional): Coordinate bounds if you wish to filter the data by location. Order should be
                                    (longitude minimum, longitude maximum, latitude minimum, latitude maximum).
    save_as (str, optional): Format to save the processed data in. For now, only 'nc' is built-in. Anything else
                                    simply returns the data.
"""
def process_Landsat_Sentinel_tif(tif, time, name, coord_bounds=None, save_as='nc'):
    #########################################################################################################
    # Open file and rename variables
    dsLS = rxr.open_rasterio(tif)
    stackedLS = dsLS.stack(stacked_dim=('x', 'y'))
    geotiff_dsLS = stackedLS.to_dataset('band')
    
    def stacked_to_latlon(pt):
        return utm_proj(pt[0], pt[1], inverse=True)

    ptsLS = list(map(stacked_to_latlon, geotiff_dsLS.stacked_dim.values))
    geotiff_dsLS.coords['latlons'] = (['points', 'values'], ptsLS)
    
    geotiff_dsLS = geotiff_dsLS.rename({1:'Landsat_Blue_Vis_Sfc_Reflectance', 2:'Landsat_Red_Vis_Sfc_Reflectance',
                                        3:'Landsat_NIR_Sfc_Reflectance', 4:'Landsat_SWIR1_Sfc_Reflectance',
                                        5:'Landsat_SWIR2_Sfc_Reflectance', 6:'Landsat_LST',
                                        7:'Landsat_Cloud_Mask', 8:'VV_SAR', 9:'VH_SAR', 10:'SAR_angle'})

    #########################################################################################################
    # Convert cloud mask integers to binary strings
    mask = xr.DataArray([[to_binary_string(x) for x in line] for line in geotiff_ds.Landsat_Cloud_Mask.values])
    mask = mask.rename({'dim_0':'stacked_dim'})
    geotiff_dsLS['Landsat_Cloud_Mask'] = mask

    #########################################################################################################
    # Assign descriptive attributes for the cloud mask and the file datetime
    geotiff_ds = geotiff_ds.assign_attrs(
        Cloud_Mask_Bits="Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high confidence)\nBit 3: Cloud\nBit 4: Cloud Shadow\n\
    Bit 5: Snow\nBit 6: Clear\n    0: Cloud or Dilated Cloud bits are set\n    1: Cloud and Dilated Cloud bits are not set\nBit 7: Water\n\
    Bits 8-9: Cloud Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 10-11: Cloud Shadow Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 12-13: Snow/Ice Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 14-15: Cirrus Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High")

    geotiff_ds = geotiff_ds.assign_attrs(Datetime=str(time))

    #########################################################################################################
    # Optional filtering by lat/lon
    if coord_bounds:
        geotiff_ds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

    #########################################################################################################
    if save_as=='nc':
        # Convert file into netCDF
        geotiff_ds.to_netcdf(f'{name}.nc')
    else:
        return geotiff_dsLS

In [67]:
"""
Processing of individual .tif files.

Performs a variety of tasks on the data to make it more easy to read and understand.

Attributes:
    tif (str): Path where tif file is located.
    time (datetime or str): Date and time of when the data was collected.
    name (str): Desired name of output file. Just the name, leave out the ".format" part at the end.
    coord_bounds (tuple or list, optional): Coordinate bounds if you wish to filter the data by location. Order should be
                                    (longitude minimum, longitude maximum, latitude minimum, latitude maximum).
    save_as (str, optional): Format to save the processed data in. For now, only 'nc' is built-in. Anything else
                                    simply returns the data.
"""
def process_GOES_tif(tif, time, name, coord_bounds=None, save_as='nc'):
    #########################################################################################################
    # Open file and rename variables
    dsG = rxr.open_rasterio(tif)
    stackedG = dsG.stack(stacked_dim=('x', 'y'))
    geotiff_dsG = stackedG.to_dataset('band')

    def stacked_to_latlon(pt):
        return utm_proj(pt[0], pt[1], inverse=True)

    ptsG = list(map(stacked_to_latlon, geotiff_dsG.stacked_dim.values))
    geotiff_dsG.coords['latlons'] = (['points', 'values'], ptsG)
    
    geotiff_dsG = geotiff_dsG.rename({1:'GOES_Blue_Vis_Reflectance',
                                        2:'GOES_Red_Vis_Reflectance', 3:'GOES_NIR_Veggie_Reflectance',
                                        4:'GOES_NIR_SnowIce_Reflectance', 5:'GOES_NIR_CloudParticle_Reflectance',
                                        6:'GOES_LWIR_Brightness_Temp', 7:'GOES_Dirty_LWIR_Brightness_temp'})

    #########################################################################################################
    # Optional filtering by lat/lon
    if coord_bounds:
        geotiff_ds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

    #########################################################################################################
    if save_as=='nc':
        # Convert file into netCDF
        geotiff_ds.to_netcdf(f'{name}.nc')
    else:
        return geotiff_ds

In [61]:
# Runs processing function on all the geotif files
#coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

for i in Landsat_indexes:
    process_Landsat_Sentinel_tif(geotiff_list[i], times_Landsat.value[i], f'../../processed_LS/tif_{i}')

In [ ]:
# Runs processing function on all the geotif files
#coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

for i in Landsat_indexes:
    process_GOES_tif(geotiff_list[i], times_GOES.value[i], f'../../processed_GOES/tif_{i}')

## Sample Code

In [2]:
dsLS = rxr.open_rasterio('../../tifs/Landsat_Sentinel_image_0.tif')
dsG = rxr.open_rasterio('../../tifs/GOES_image_0.tif')
#ds1 = rxr.open_rasterio('../../tifs/GOES_image_0.tif')
#ds2 = rxr.open_rasterio('../../tifs/GOES_image_1.tif')

In [35]:
stackedLS = dsLS.stack(stacked_dim=('x', 'y'))
stackedG = dsG.stack(stacked_dim=('x', 'y'))
stackedLS

<xarray.DataArray (band: 10, stacked_dim: 9000000)>
array([[  0.0749175 ,   0.069225  ,   0.072745  , ...,   0.1472975 ,
          0.144685  ,   0.145785  ],
       [  0.11251   ,   0.127965  ,   0.1464725 , ...,   0.2026    ,
          0.20172   ,   0.204635  ],
       [  0.2763    ,   0.28697   ,   0.3068525 , ...,   0.2268275 ,
          0.22856   ,   0.2327125 ],
       ...,
       [         nan,          nan,          nan, ..., -21.50701697,
        -22.10128354, -23.15400349],
       [         nan,          nan,          nan, ..., -31.05807109,
        -33.45023946, -32.93874892],
       [         nan,          nan,          nan, ...,  43.37388992,
         43.37364578,  43.37339783]])
Coordinates:
  * band         (band) int64 1 2 3 4 5 6 7 8 9 10
    spatial_ref  int64 0
  * stacked_dim  (stacked_dim) object MultiIndex
  * x            (stacked_dim) float64 2.92e+05 2.92e+05 ... 3.82e+05 3.82e+05
  * y            (stacked_dim) float64 4.372e+06 4.372e+06 ... 4.282e+06
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [36]:
geotiff_dsLS = stackedLS.to_dataset('band')
geotiff_dsG = stackedG.to_dataset('band')
#geotiff_ds1 = ds1.to_dataset('band')
#geotiff_ds2 = ds2.to_dataset('band')

In [38]:
def stacked_to_latlon(pt):
    return utm_proj(pt[0], pt[1], inverse=True)

In [39]:
ptsLS = list(map(stacked_to_latlon, geotiff_dsLS.stacked_dim.values))
ptsG = list(map(stacked_to_latlon, geotiff_dsG.stacked_dim.values))

In [41]:
geotiff_dsLS.coords['latlons'] = (['points', 'values'], ptsLS)
geotiff_dsG.coords['latlons'] = (['points', 'values'], ptsG)

In [42]:
geotiff_dsLS

<xarray.Dataset>
Dimensions:      (stacked_dim: 9000000, points: 9000000, values: 2)
Coordinates:
    spatial_ref  int64 0
  * stacked_dim  (stacked_dim) object MultiIndex
  * x            (stacked_dim) float64 2.92e+05 2.92e+05 ... 3.82e+05 3.82e+05
  * y            (stacked_dim) float64 4.372e+06 4.372e+06 ... 4.282e+06
    latlons      (points, values) float64 -77.42 39.47 -77.42 ... -76.36 38.68
Dimensions without coordinates: points, values
Data variables:
    1            (stacked_dim) float64 0.07492 0.06922 0.07275 ... 0.1447 0.1458
    2            (stacked_dim) float64 0.1125 0.128 0.1465 ... 0.2017 0.2046
    3            (stacked_dim) float64 0.2763 0.287 0.3069 ... 0.2286 0.2327
    4            (stacked_dim) float64 0.2057 0.2238 0.2356 ... 0.1354 0.1351
    5            (stacked_dim) float64 0.1595 0.1727 0.1785 ... 0.1408 0.1401
    6            (stacked_dim) float64 262.5 262.5 262.5 ... 246.8 246.7 246.7
    7            (stacked_dim) float64 5.505e+04 5.505e+04 ... 5.505e+04
    8            (stacked_dim) float64 nan nan nan nan ... -21.51 -22.1 -23.15
    9            (stacked_dim) float64 nan nan nan nan ... -31.06 -33.45 -32.94
    10           (stacked_dim) float64 nan nan nan nan ... 43.37 43.37 43.37
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [43]:
geotiff_dsG

<xarray.Dataset>
Dimensions:      (stacked_dim: 9000000, points: 9000000, values: 2)
Coordinates:
    spatial_ref  int64 0
  * stacked_dim  (stacked_dim) object MultiIndex
  * x            (stacked_dim) float64 2.92e+05 2.92e+05 ... 3.82e+05 3.82e+05
  * y            (stacked_dim) float64 4.372e+06 4.372e+06 ... 4.282e+06
    latlons      (points, values) float64 -77.42 39.47 -77.42 ... -76.36 38.68
Dimensions without coordinates: points, values
Data variables:
    1            (stacked_dim) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    2            (stacked_dim) float64 0.0002442 0.0002442 0.0002442 ... 0.0 0.0
    3            (stacked_dim) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    4            (stacked_dim) float64 0.0002442 0.0002442 ... 0.0004884
    5            (stacked_dim) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    6            (stacked_dim) float64 293.3 293.3 293.3 ... 295.8 295.8 295.8
    7            (stacked_dim) float64 292.3 292.3 292.3 ... 294.7 294.7 294.7
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06', '...

In [44]:
# For GOES tifs
geotiff_dsG = geotiff_dsG.rename({1:'GOES_Blue_Vis_Reflectance',
                               2:'GOES_Red_Vis_Reflectance', 3:'GOES_NIR_Veggie_Reflectance',
                               4:'GOES_NIR_SnowIce_Reflectance', 5:'GOES_NIR_CloudParticle_Reflectance',
                               6:'GOES_LWIR_Brightness_Temp', 7:'GOES_Dirty_LWIR_Brightness_temp'})

In [45]:
# For Landsat/Sentinel tifs
geotiff_dsLS = geotiff_dsLS.rename({1:'Landsat_Blue_Vis_Sfc_Reflectance', 2:'Landsat_Red_Vis_Sfc_Reflectance',
                                3:'Landsat_NIR_Sfc_Reflectance', 4:'Landsat_SWIR1_Sfc_Reflectance',
                               5:'Landsat_SWIR2_Sfc_Reflectance', 6:'Landsat_LST',
                               7:'Landsat_Cloud_Mask', 8:'VV_SAR', 9:'VH_SAR', 10:'SAR_angle'})

In [66]:
geotiff_dsG

<xarray.Dataset>
Dimensions:                             (stacked_dim: 9000000, points: 9000000,
                                         values: 2)
Coordinates:
    spatial_ref                         int64 0
  * stacked_dim                         (stacked_dim) object MultiIndex
  * x                                   (stacked_dim) float64 2.92e+05 ... 3....
  * y                                   (stacked_dim) float64 4.372e+06 ... 4...
    latlons                             (points, values) float64 -77.42 ... 3...
Dimensions without coordinates: points, values
Data variables:
    GOES_Blue_Vis_Reflectance           (stacked_dim) float64 0.0 0.0 ... 0.0
    GOES_Red_Vis_Reflectance            (stacked_dim) float64 0.0002442 ... 0.0
    GOES_NIR_Veggie_Reflectance         (stacked_dim) float64 0.0 0.0 ... 0.0
    GOES_NIR_SnowIce_Reflectance        (stacked_dim) float64 0.0002442 ... 0...
    GOES_NIR_CloudParticle_Reflectance  (stacked_dim) float64 0.0 0.0 ... 0.0
    GOES_LWIR_Brightness_Temp           (stacked_dim) float64 293.3 ... 295.8
    GOES_Dirty_LWIR_Brightness_temp     (stacked_dim) float64 292.3 ... 294.7
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06', '...

In [48]:
# Baltimore-ish
coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

In [49]:
#gds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

In [54]:
mask = xr.DataArray([to_binary_string(x) for x in geotiff_dsLS.Landsat_Cloud_Mask.values])

In [55]:
mask

<xarray.DataArray (dim_0: 9000000)>
array(['1101011100001100', '1101011100001100', '1101011100001100', ...,
       '1101011100001100', '1101011100001100', '1101011100001100'],
      dtype='<U16')
Dimensions without coordinates: dim_0

In [48]:
water = np.array([[s[-8] for s in line] for line in mask.values])

In [49]:
wmask = (water == '0')*1

In [50]:
np.sum((geotiff_ds.Landsat_LST)*wmask == 0)

<xarray.DataArray 'Landsat_LST' ()>
array(993622)
Coordinates:
    spatial_ref  int64 0

In [56]:
mask = mask.rename({'dim_0':'stacked_dim'})

In [57]:
geotiff_dsLS['Landsat_Cloud_Mask'] = mask

In [58]:
geotiff_dsLS

<xarray.Dataset>
Dimensions:                           (stacked_dim: 9000000, points: 9000000,
                                       values: 2)
Coordinates:
    spatial_ref                       int64 0
  * stacked_dim                       (stacked_dim) object MultiIndex
  * x                                 (stacked_dim) float64 2.92e+05 ... 3.82...
  * y                                 (stacked_dim) float64 4.372e+06 ... 4.2...
    latlons                           (points, values) float64 -77.42 ... 38.68
Dimensions without coordinates: points, values
Data variables:
    Landsat_Blue_Vis_Sfc_Reflectance  (stacked_dim) float64 0.07492 ... 0.1458
    Landsat_Red_Vis_Sfc_Reflectance   (stacked_dim) float64 0.1125 ... 0.2046
    Landsat_NIR_Sfc_Reflectance       (stacked_dim) float64 0.2763 ... 0.2327
    Landsat_SWIR1_Sfc_Reflectance     (stacked_dim) float64 0.2057 ... 0.1351
    Landsat_SWIR2_Sfc_Reflectance     (stacked_dim) float64 0.1595 ... 0.1401
    Landsat_LST                       (stacked_dim) float64 262.5 ... 246.7
    Landsat_Cloud_Mask                (stacked_dim) <U16 '1101011100001100' ....
    VV_SAR                            (stacked_dim) float64 nan nan ... -23.15
    VH_SAR                            (stacked_dim) float64 nan nan ... -32.94
    SAR_angle                         (stacked_dim) float64 nan nan ... 43.37
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [60]:
geotiff_dsLS = geotiff_dsLS.assign_attrs(
    Cloud_Mask_Bits="Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high confidence)\nBit 3: Cloud\nBit 4: Cloud Shadow\n\
Bit 5: Snow\nBit 6: Clear\n    0: Cloud or Dilated Cloud bits are set\n    1: Cloud and Dilated Cloud bits are not set\nBit 7: Water\n\
Bits 8-9: Cloud Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 10-11: Cloud Shadow Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 12-13: Snow/Ice Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 14-15: Cirrus Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High")

In [61]:
geotiff_dsLS = geotiff_dsLS.assign_attrs(Datetime=str(times.value[0]))

NameError: name 'times' is not defined

In [62]:
geotiff_dsLS

<xarray.Dataset>
Dimensions:                           (stacked_dim: 9000000, points: 9000000,
                                       values: 2)
Coordinates:
    spatial_ref                       int64 0
  * stacked_dim                       (stacked_dim) object MultiIndex
  * x                                 (stacked_dim) float64 2.92e+05 ... 3.82...
  * y                                 (stacked_dim) float64 4.372e+06 ... 4.2...
    latlons                           (points, values) float64 -77.42 ... 38.68
Dimensions without coordinates: points, values
Data variables:
    Landsat_Blue_Vis_Sfc_Reflectance  (stacked_dim) float64 0.07492 ... 0.1458
    Landsat_Red_Vis_Sfc_Reflectance   (stacked_dim) float64 0.1125 ... 0.2046
    Landsat_NIR_Sfc_Reflectance       (stacked_dim) float64 0.2763 ... 0.2327
    Landsat_SWIR1_Sfc_Reflectance     (stacked_dim) float64 0.2057 ... 0.1351
    Landsat_SWIR2_Sfc_Reflectance     (stacked_dim) float64 0.1595 ... 0.1401
    Landsat_LST                       (stacked_dim) float64 262.5 ... 246.7
    Landsat_Cloud_Mask                (stacked_dim) <U16 '1101011100001100' ....
    VV_SAR                            (stacked_dim) float64 nan nan ... -23.15
    VH_SAR                            (stacked_dim) float64 nan nan ... -32.94
    SAR_angle                         (stacked_dim) float64 nan nan ... 43.37
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...

In [ ]:
# Export the data
#geotiff_ds.rio.to_raster("first.tif")
geotiff_ds.to_netcdf('first.nc')

In [27]:
xr.load_dataset('first.nc')

<xarray.Dataset>
Dimensions:                             (latitude: 6400, longitude: 6400)
Coordinates:
  * longitude                           (longitude) float64 -77.51 ... -75.79
  * latitude                            (latitude) float64 39.9 39.9 ... 38.17
Data variables: (12/15)
    Landsat_Blue_Vis_Sfc_Reflectance    (latitude, longitude) float64 nan ......
    Landsat_Red_Vis_Sfc_Reflectance     (latitude, longitude) float64 nan ......
    Landsat_NIR_Sfc_Reflectance         (latitude, longitude) float64 nan ......
    Landsat_SWIR1_Sfc_Reflectance       (latitude, longitude) float64 nan ......
    Landsat_SWIR2_Sfc_Reflectance       (latitude, longitude) float64 nan ......
    Landsat_LST                         (latitude, longitude) float64 nan ......
    ...                                  ...
    GOES_NIR_Veggie_Reflectance         (latitude, longitude) float64 0.1299 ...
    GOES_NIR_SnowIce_Reflectance        (latitude, longitude) float64 0.06935...
    GOES_NIR_CloudParticle_Reflectance  (latitude, longitude) float64 0.06691...
    GOES_LWIR_Brightness_Temp           (latitude, longitude) float64 274.2 ....
    GOES_Dirty_LWIR_Brightness_temp     (latitude, longitude) float64 273.1 ....
    spatial_ref                         int64 0
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:33.089500

In [ ]:
######################################################
# End of tif processing part
# Below is gridding of Baltimore

In [31]:
grid_lats = [39.2576, 39.3147, 39.3718]
grid_lons = [-76.5299, -76.6204, -76.7108]

In [37]:
grid1 = geotiff_ds.sel(longitude=slice(grid_lons[2], grid_lons[1])).sel(latitude=slice(grid_lats[2], grid_lats[1]))
grid2 = geotiff_ds.sel(longitude=slice(grid_lons[1], grid_lons[0])).sel(latitude=slice(grid_lats[2], grid_lats[1]))
grid3 = geotiff_ds.sel(longitude=slice(grid_lons[2], grid_lons[1])).sel(latitude=slice(grid_lats[1], grid_lats[0]))
grid4 = geotiff_ds.sel(longitude=slice(grid_lons[1], grid_lons[0])).sel(latitude=slice(grid_lats[1], grid_lats[0]))

In [38]:
grid4

<xarray.Dataset>
Dimensions:                             (latitude: 212, longitude: 336)
Coordinates:
  * longitude                           (longitude) float64 -76.62 ... -76.53
  * latitude                            (latitude) float64 39.31 39.31 ... 39.26
    spatial_ref                         int64 0
Data variables: (12/14)
    Landsat_Blue_Vis_Sfc_Reflectance    (latitude, longitude) float64 ...
    Landsat_Red_Vis_Sfc_Reflectance     (latitude, longitude) float64 ...
    Landsat_NIR_Sfc_Reflectance         (latitude, longitude) float64 ...
    Landsat_SWIR1_Sfc_Reflectance       (latitude, longitude) float64 ...
    Landsat_SWIR2_Sfc_Reflectance       (latitude, longitude) float64 ...
    Landsat_LST                         (latitude, longitude) float64 ...
    ...                                  ...
    GOES_Red_Vis_Reflectance            (latitude, longitude) float64 ...
    GOES_NIR_Veggie_Reflectance         (latitude, longitude) float64 ...
    GOES_NIR_SnowIce_Reflectance        (latitude, longitude) float64 ...
    GOES_NIR_CloudParticle_Reflectance  (latitude, longitude) float64 ...
    GOES_LWIR_Brightness_Temp           (latitude, longitude) float64 ...
    GOES_Dirty_LWIR_Brightness_temp     (latitude, longitude) float64 ...
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:33.089500